
# Fantasy Premier League - Player Selector
# Navankur Verma - navankurverma@gmail.com

[(Parent link)](https://navankurverma.github.io/MyFPL/Navankur_Verma_FPL.html)

Libraries Used:

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import plotly.express as px
%matplotlib inline

### Pull General FPL Data 
[Move to top](#Fantasy-Premier-League---Player-Selector)

In [2]:
#Whole Data
fpl = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
fpl = fpl.json()

In [3]:
if 'teams' in fpl:
    teams = pd.DataFrame(fpl['teams'])
    teams = teams.iloc[:,[0,3,5,9,10,14,15,16,17,18,19]] #picking only valuable columns
    teams.rename(columns = {'code': 'team_code', 'id': 'team_id', 'name': 'team_name'}, inplace = True)
    teams
else:
    print('Teams data not found')

In [4]:
if 'element_types' in fpl:
    element_types = pd.DataFrame(fpl['element_types'])
    element_types.rename(columns= {'id': 'element_type'}, inplace = True)
    element_types
else:
    print('Element_types data not found')

In [5]:
if 'elements' in fpl:
    players = fpl['elements']
    players = pd.DataFrame(players)
else:
    print('Players data not found')

#### Cleaning Public Data
[Move to top](#Fantasy-Premier-League---Player-Selector)

In [6]:
players.isnull().sum()

chance_of_playing_next_round           236
chance_of_playing_this_round           243
code                                     0
cost_change_event                        0
cost_change_event_fall                   0
                                      ... 
corners_and_indirect_freekicks_text      0
direct_freekicks_order                 554
direct_freekicks_text                    0
penalties_order                        565
penalties_text                           0
Length: 67, dtype: int64

In [7]:
#Replacing missing values with Zeroes
players['chance_of_playing_next_round'] = players['chance_of_playing_next_round'].fillna(0)
players['chance_of_playing_this_round'] = players['chance_of_playing_this_round'].fillna(0)

In [8]:
players.isnull().sum()

chance_of_playing_next_round             0
chance_of_playing_this_round             0
code                                     0
cost_change_event                        0
cost_change_event_fall                   0
                                      ... 
corners_and_indirect_freekicks_text      0
direct_freekicks_order                 554
direct_freekicks_text                    0
penalties_order                        565
penalties_text                           0
Length: 67, dtype: int64

Column Description by its index:  
14,12,35,26,27,8 <- id, name, teamName, position  
29,30,31,32 <- Transfer details  
25,0,1 <- Chances of playing  
7, 11,13,15,20,28,18,22 <- fpl related details  
36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52 <- onfield stats  
Selecting only above columns:

In [9]:
players.rename(columns = {'id': 'element'}, inplace = True)
players_details = players.iloc[:,[14,12,35,26,27,8, 36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52, 7,11,13,15,20,28,18,22,29,30,31,32,25,0,1]]

Joining players data with their corresponding details and teams:

In [10]:
players_details = players_details.merge(teams[["team_code","team_name"]],
                                        on = "team_code").merge(element_types[["singular_name","element_type"]],
                                                                on = "element_type")
players_details.drop(['team','team_code'], axis=1, inplace = True)
players_details.set_index('element', inplace = True)

In [11]:
players_details.head()

,first_name,web_name,element_type,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,...,selected_by_percent,transfers_in,transfers_in_event,transfers_out,transfers_out_event,status,chance_of_playing_next_round,chance_of_playing_this_round,team_name,singular_name
element,,,,,,,,,,,,,,,,,,,,,
1,Bernd,Leno,1,270,0,0,0,9,0,0,...,1.0,59562,133,172255,704,a,0.0,0.0,Arsenal,Goalkeeper
2,Rúnar Alex,Rúnarsson,1,0,0,0,0,0,0,0,...,0.7,19017,0,64604,305,u,0.0,0.0,Arsenal,Goalkeeper
532,Karl,Hein,1,0,0,0,0,0,0,0,...,0.5,46353,86,23796,295,a,0.0,0.0,Arsenal,Goalkeeper
559,Aaron,Ramsdale,1,900,0,0,6,8,0,0,...,12.7,1158766,34685,166200,5675,a,0.0,0.0,Arsenal,Goalkeeper
572,Arthur,Okonkwo,1,0,0,0,0,0,0,0,...,0.1,13344,35,6804,92,a,0.0,0.0,Arsenal,Goalkeeper


#### Most Suitable Player to Buy Currently
[Move to top](#Fantasy-Premier-League---Player-Selector)

In [12]:
cf.go_offline()

df = players_details[['form','points_per_game','now_cost','team_name','singular_name','first_name','web_name']]
df["now_cost"] = df["now_cost"]/10
df.points_per_game = pd.to_numeric(df.points_per_game)
df.rename(columns = {'singular_name' : "Player Type"}, inplace = True)
df["name"] = df["first_name"] + ' ' + df["web_name"]
fig = px.scatter(df, x='points_per_game', 
                 y='now_cost',  
                 size = (players_details["form"].astype(float).values + abs(players_details["form"].astype(float).values.min()))*1.5, 
                 hover_name = "name",
                 hover_data= ["team_name"],
                 color ="Player Type",
                 title = "Most Suitable Player to Buy Currently"
                )
fig['layout']['yaxis'].update({'title': 'Price', 'tickprefix': '£'})
fig['layout']['xaxis'].update({'title': 'Points Per Game'})
fig.update_layout(
    showlegend=True,
    annotations=[
        dict(
            x=1.15,
            y=1.10,
            xref="paper",
            yref="paper",
            showarrow=False,
            text="Bubble Size : Player Form"
        )
    ]
)

fig.show()